# Iterative Proportional Fitting Code Demo


Code from https://github.com/Dirguis/ipfn (IPFN package for Python).

In [227]:
from ipfn import ipfn
import numpy as np
import pandas as pd

### Input Description

Variable      | Description   | Type
------------- | ------------- | -------------
m  | Original matrix on which to perform IPF, initialize all cells | Numpy matrix
Marginals  | Target sums when aggreating along specified axis | List of Numpy arrays
Dimensions  | Axes along which to sum to get marginals | List of lists of integers

### Import Census Data

In [228]:
# excel sheet preselected for age, race, and renter/owner and including only Morningside Heights
census_data_selected = pd.read_excel('planning_database_NYC_tracts_numbers.xlsx')
census_data_selected

,Tract,Neighborhood,Tot_Population_CEN_2010,Pop_under_5_CEN_2010,Pop_5_17_CEN_2010,Pop_18_24_CEN_2010,Pop_25_44_CEN_2010,Pop_45_64_CEN_2010,Pop_65plus_CEN_2010,Hispanic_CEN_2010,NH_White_alone_CEN_2010,NH_Blk_alone_CEN_2010,NH_AIAN_alone_CEN_2010,NH_Asian_alone_CEN_2010,NH_NHOPI_alone_CEN_2010,NH_SOR_alone_CEN_2010,Pop_Multiple,Renter_Occp_HU_CEN_2010,Owner_Occp_HU_CEN_2010
0,19701,Morningside Heights,641,42,59,58,331,116,35,90,331,102,2,100,0,4,12,641,0
1,19900,Morningside Heights,10064,316,660,2974,3105,1711,1298,1315,6001,655,12,1692,7,25,357,7392,2672
2,20101,Morningside Heights,1731,36,54,1153,269,139,80,211,907,143,4,382,2,7,75,1477,254
3,20300,Morningside Heights,3633,62,90,2210,1076,120,75,512,1713,354,6,877,3,9,159,3502,131
4,20500,Morningside Heights,5322,173,305,2419,1006,879,540,529,3428,347,11,798,4,32,173,4348,974
5,20701,Morningside Heights,3329,113,107,760,1765,375,209,342,2015,181,4,659,3,15,110,3191,138
6,20901,Morningside Heights,3673,212,692,455,931,857,526,1632,164,1664,4,127,1,13,68,3499,174
7,21100,Morningside Heights,10330,430,1084,1483,3669,2275,1389,2734,3829,2099,35,1334,2,36,261,7645,2685


In [229]:
# total population per tract
df_tot = census_data_selected['Tot_Population_CEN_2010']
df_tot

0      641
1    10064
2     1731
3     3633
4     5322
5     3329
6     3673
7    10330
Name: Tot_Population_CEN_2010, dtype: int64

In [230]:
# renter owner marginals per tract
df_rent = census_data_selected[['Renter_Occp_HU_CEN_2010', 'Owner_Occp_HU_CEN_2010']]
df_rent

,Renter_Occp_HU_CEN_2010,Owner_Occp_HU_CEN_2010
0,641,0
1,7392,2672
2,1477,254
3,3502,131
4,4348,974
5,3191,138
6,3499,174
7,7645,2685


In [231]:
# race marginals per tract
df_race = census_data_selected[['Hispanic_CEN_2010','NH_White_alone_CEN_2010','NH_Blk_alone_CEN_2010','NH_AIAN_alone_CEN_2010','NH_Asian_alone_CEN_2010','NH_NHOPI_alone_CEN_2010','NH_SOR_alone_CEN_2010', 'Pop_Multiple']]
df_race

,Hispanic_CEN_2010,NH_White_alone_CEN_2010,NH_Blk_alone_CEN_2010,NH_AIAN_alone_CEN_2010,NH_Asian_alone_CEN_2010,NH_NHOPI_alone_CEN_2010,NH_SOR_alone_CEN_2010,Pop_Multiple
0,90,331,102,2,100,0,4,12
1,1315,6001,655,12,1692,7,25,357
2,211,907,143,4,382,2,7,75
3,512,1713,354,6,877,3,9,159
4,529,3428,347,11,798,4,32,173
5,342,2015,181,4,659,3,15,110
6,1632,164,1664,4,127,1,13,68
7,2734,3829,2099,35,1334,2,36,261


### Initialize Original Matrix

In [232]:
m = np.ones((len(df_race), len(df_race.columns), 2))

### Define Marginals and Corresponding Dimensions

In [233]:
xipp = np.array(df_tot)
xijp = np.array(df_race)
xipk = np.array(df_rent)
xpjp = xijp.sum(axis=0)
xppk = xipk.sum(axis=0)

In [234]:
aggregates = [xipp, xpjp, xppk, xijp, xipk]
dimensions = [[0], [1], [2], [0, 1], [0, 2]]

### Run IPF

In [235]:
IPF = ipfn.ipfn(m, aggregates, dimensions)
M = IPF.iteration()
M = np.array(M)

/Users/noniethomas/Library/Python/3.7/lib/python/site-packages/ipfn/ipfn.py:129: RuntimeWarning: invalid value encountered in double_scalars
  if abs(m_ijk / ori_ijk - 1) > max_conv:


### Error Analysis: Compare Actual Marginal to Desired

In [236]:
error_count = 0
tolerance = 1e-12

for i in range(M.shape[0]):
    if abs(M[i,:,:].sum() - xipp[i]) > tolerance:
        error_count += 1
    for j in range(M.shape[1]):
        if abs(M[i, j, :].sum() - xijp[i, j]) > tolerance:
            error_count += 1
    for k in range(M.shape[2]):
        if abs(M[i, :, k].sum() - xipk[i, k]) > tolerance:
            error_count += 1
for j in range(M.shape[1]):
    if abs(M[:,j,:].sum() - xpjp[j]) > tolerance:
        error_count += 1
for k in range(M.shape[2]):
    if abs(M[:,:,k].sum() - xppk[k]) > tolerance:
        error_count += 1

error_count

0

### Experiment With Initial Values

In [237]:
m_2 = np.zeros((len(df_race), len(df_race.columns), 2)) + 1e-8
m_3 = np.zeros((len(df_race), len(df_race.columns), 2)) + 1e8

In [238]:
IPF_2 = ipfn.ipfn(m_2, aggregates, dimensions)
IPF_3 = ipfn.ipfn(m_3, aggregates, dimensions)
M_2 = IPF_2.iteration()
M_3 = IPF_3.iteration()
M_2 = np.array(M_2)
M_3 = np.array(M_3)

/Users/noniethomas/Library/Python/3.7/lib/python/site-packages/ipfn/ipfn.py:129: RuntimeWarning: invalid value encountered in double_scalars
  if abs(m_ijk / ori_ijk - 1) > max_conv:


In [239]:
tolerance = 1e-16
M2_error_count = 0
M2_error = []
M3_error_count = 0
M3_error = []

for i in range(M.shape[0]):
    for j in range(M.shape[1]):
        for k in range(M.shape[2]):
            if abs(M_2[i, j, k] - M[i, j, k]) > tolerance:
                M2_error_count += 1
                M2_error.append(abs(M_2[i, j, k] - M[i, j, k]))
            if abs(M_3[i, j, k] - M[i, j, k]) > tolerance:
                M3_error_count += 1
                M3_error.append(abs(M_3[i, j, k] - M[i, j, k]))

                
print('M_2 error count: ' + str(M2_error_count))
if M2_error_count != 0:
    print('Average M_2 error size ' + str(np.mean(np.array(M2_error))))
print('M_3 error count: ' + str(M3_error_count))
if M3_error_count != 0:
    print('Average M_3 error size ' + str(np.mean(np.array(M3_error))))

M_2 error count: 8
Average M_2 error size 1.4072076837123859e-14
M_3 error count: 0


### Example Tract

In [240]:
tract3 = M[2]
tract3.transpose()

array([[1.80038706e+02, 7.73910456e+02, 1.22016753e+02, 3.41305604e+00,
        3.25946852e+02, 1.70652802e+00, 5.97284806e+00, 6.39948007e+01],
       [3.09612940e+01, 1.33089544e+02, 2.09832467e+01, 5.86943963e-01,
        5.60531485e+01, 2.93471982e-01, 1.02715194e+00, 1.10051993e+01]])